In [1]:
%pip install flask transformers torch 

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install sentencepiece --quiet

In [3]:
!python -m pip install --upgrade pip

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----------- ---------------------------- 0.5/1.8 MB 1.4 MB/s eta 0:00:01
   ---------------------------- ----------- 1.3/1.8 MB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 1.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.1
    Uninstalling pip-25.1:
      Successfully uninstalled pip-25.1


In [14]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load tokenizer
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")

# Load model in TensorFlow
model_pt = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")

# Example: encode some text and get a prediction
input_text = "Translate English to French: My name is Zainab!"
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model_pt.generate(**inputs)
translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Translated text (PyTorch):", translated_text)

Translated text (PyTorch): J'ai le nom de Zainab!


In [5]:
%pip install flask-cors

In [15]:
from flask import Flask, request, jsonify
from transformers import pipeline
import torch
from flask_cors import CORS
import threading
import time

# Check for GPU availability and set device
device = 0 if torch.cuda.is_available() else -1
print(f"Device set to use {'gpu' if device == 0 else 'cpu'}")

# Load the translation model
translator = pipeline("text2text-generation", model="google/flan-t5-large", device=device)

# Initialize Flask
app = Flask(__name__)
# Enable CORS for all routes with specific configuration
# Configure CORS with specific settings
CORS(app, 
     resources={r"/*": {
         "origins": "*",
         "methods": ["GET", "POST", "OPTIONS"],
         "allow_headers": ["Content-Type", "Authorization", "Accept"],
         "expose_headers": ["Content-Type", "Authorization"],
         "supports_credentials": True,
     }},
     supports_credentials=True)

@app.after_request
def after_request(response):
    # Add security headers
    response.headers.add('Cross-Origin-Opener-Policy', 'same-origin-allow-popups')
    response.headers.add('Cross-Origin-Embedder-Policy', 'require-corp')
    response.headers.add('Cross-Origin-Resource-Policy', 'cross-origin')
    return response
@app.route('/')
def home():
    return "🟢 Translation API is running!"

@app.route('/translate', methods=['POST', 'OPTIONS'])
def translate_text():
    # Handle preflight request
    if request.method == 'OPTIONS':
        response = jsonify({'status': 'ok'})
        return response


    try:
        # Get request data
        data = request.get_json()

        # Validate input
        text = data.get('text', '').strip()
        if not text:
            return jsonify({'error': 'Text must be a non-empty string'}), 400

        source_lang = data.get('source_lang', 'English').strip()
        target_lang = data.get('target_lang', 'French').strip()

        if not source_lang or not target_lang:
            return jsonify({'error': 'Both source_lang and target_lang must be provided'}), 400

        # Create dynamic prompt
        prompt = f"translate {source_lang} to {target_lang}: {text}"

        # Translate
        translated_text = translator(prompt, max_length=256, do_sample=False)

        response = jsonify({
            'translation': translated_text[0]['generated_text'],
            'source_lang': source_lang,
            'target_lang': target_lang
        })
        response.headers.add('Access-Control-Allow-Origin', '*')
        return response

    except Exception as e:
        response = jsonify({'error': str(e)}), 500
        response[0].headers.add('Access-Control-Allow-Origin', '*')
        return response

def run_flask():
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

# Run Flask app in a separate thread
flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

time.sleep(2)  # Allow server time to start

Device set to use cpu


Device set to use cpu


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://169.254.95.18:5000
Press CTRL+C to quit
